In [1]:
import numpy as np
import pandas as pd
import pulp

class DEAProblem:
    def __init__(
        self,
        inputs,
        outputs,
        bad_outs,
        weight_vector,
        directional_factor=None,
        returns="CRS",
        disp="weak disposability",
        in_weights=[0, None],
        out_weights=[0, None],
        badout_weights=[0, None],
    ):
        self.inputs = inputs
        self.outputs = outputs
        self.bad_outs = bad_outs
        self.returns = returns
        self.weight_vector = (
            weight_vector  # weight vector in directional distance function
        )
        self.disp = disp

        self.J, self.I = self.inputs.shape  # no of DMUs, inputs
        _, self.R = self.outputs.shape  # no of outputs
        _, self.S = self.bad_outs.shape  # no of bad outputs
        self._i = range(self.I)  # inputs
        self._r = range(self.R)  # outputs
        self._s = range(self.S)  # bad_output
        self._j = range(self.J)  # DMUs
        if directional_factor == None:
            self.gx = self.inputs
            self.gy = self.outputs
            self.gb = self.bad_outs
        else:
            self.gx = directional_factor[: self.I]
            self.gy = directional_factor[self.I : (self.I + self.J)]
            self.gy = directional_factor[(self.I + self.J) :]

        self._in_weights = in_weights  # input weight restrictions
        self._out_weights = out_weights  # output weight restrictions
        self._badout_weights = badout_weights  # bad output weight restrictions

        # creates dictionary of pulp.LpProblem objects for the DMUs
        self.dmus = self._create_problems()

    def _create_problems(self):
        """
        Iterate over the DMU and create a dictionary of LP problems, one
        for each DMU.
        """

        dmu_dict = {}
        for j0 in self._j:
            dmu_dict[j0] = self._make_problem(j0)
        return dmu_dict

    def _make_problem(self, j0):
        """
        Create a pulp.LpProblem for a DMU.
        """

        # Set up pulp
        prob = pulp.LpProblem("".join(["DMU_", str(j0)]), pulp.LpMaximize)
        self.weights = pulp.LpVariable.dicts(
            "Weight", (self._j), lowBound=self._in_weights[0]
        )
        self.betax = pulp.LpVariable.dicts(
            "scalingFactor_x", (self._i), lowBound=0, upBound=1
        )

        self.betay = pulp.LpVariable.dicts("scalingFactor_y", (self._r), lowBound=0)

        self.betab = pulp.LpVariable.dicts(
            "scalingFactor_b", (self._s), lowBound=0, upBound=1
        )

        # Set returns to scale
        if self.returns == "VRS":
            prob += pulp.lpSum([self.weights[j] for j in self.weights]) == 1

        # Set up objective function
        prob += pulp.lpSum(
            [(self.weight_vector[i] * self.betax[i]) for i in self._i]
            + [(self.weight_vector[self.I + r] * self.betay[r]) for r in self._r]
            + [
                (self.weight_vector[self.I + self.R + s] * self.betab[s])
                for s in self._s
            ]
        )

        # Set up constraints
        for i in self._i:
            prob += (
                pulp.lpSum(
                    [(self.weights[j0] * self.inputs.values[j0][i]) for j0 in self._j]
                )
                <= self.inputs.values[j0][i] - self.betax[i] * self.gx.values[j0][i]
            )
        for r in self._r:
            prob += (
                pulp.lpSum(
                    [(self.weights[j0] * self.outputs.values[j0][r]) for j0 in self._j]
                )
                >= self.outputs.values[j0][r] + self.betay[r] * self.gy.values[j0][r]
            )

        if self.disp == "weak disposability":
            for s in self._s:  # weak disposability
                prob += (
                    pulp.lpSum(
                        [
                            (self.weights[j0] * self.bad_outs.values[j0][s])
                            for j0 in self._j
                        ]
                    )
                    == self.bad_outs.values[j0][s]
                    - self.betab[s] * self.gb.values[j0][s]
                )

        elif self.disp == "strong disposability":
            for s in self._s:  # strong disposability
                prob += (
                    pulp.lpSum(
                        [
                            (self.weights[j0] * self.bad_outs.values[j0][s])
                            for j0 in self._j
                        ]
                    )
                    >= self.bad_outs.values[j0][s]
                    - self.betab[s] * self.gb.values[j0][s]
                )

        return prob

    def solve(self):
        """
        Iterate over the dictionary of DMUs' problems, solve them, and collate
        the results into a pandas dataframe.
        """

        sol_status = {}
        sol_weights = {}
        sol_efficiency = {}

        for ind, problem in list(self.dmus.items()):
            problem.solve()
            sol_status[ind] = pulp.LpStatus[problem.status]
            sol_weights[ind] = {}
            for v in problem.variables():
                sol_weights[ind][v.name] = v.varValue
            sol_efficiency[ind] = pulp.value(problem.objective)
        return sol_status, sol_efficiency, sol_weights

In [2]:
#pip install -i https://pypi.tuna.tsinghua.edu.cn/simple  pulp    

In [2]:
X = pd.DataFrame(
    np.array(
        [
            [20, 300,20],
            [30, 200,30],
            [40, 100,40],
            [20, 200,20],
            [10, 400,10],
            [11, 222,11],
            [12, 321,12],
            [14, 231,14],
        ]
    )
)
y = pd.DataFrame(np.array([[20], [30], [40], [30], [50], [21], [32], [42]]))
b = pd.DataFrame(np.array([[10], [20], [10], [10], [10], [12], [-2], [-1]]))
weight = [1 / 9, 1 / 9,1 / 9 ,1 / 3, 1 / 3]
names = pd.DataFrame(
    ["Bratislava", "Zilina", "Kosice", "Presov", "Poprad", "ala", "ba", "ca"],
    columns=["DMU"],
)

solve = DEAProblem(X, y, b, weight, disp="weak disposability").solve()

status = pd.DataFrame.from_dict(solve[0], orient="index", columns=["status"])
efficiency = pd.DataFrame.from_dict(solve[1], orient="index", columns=["efficiency"])
weights = pd.DataFrame.from_dict(solve[2], orient="index")
results = pd.concat([names, status, efficiency, weights], axis=1)

In [3]:
print(results.round(decimals=2))####保留round(decimals=2)几位小数

          DMU   status  efficiency  Weight_0  Weight_1  Weight_2  Weight_3  \
0  Bratislava  Optimal        0.92       0.0       0.0      0.07       0.0   
1      Zilina  Optimal        0.54       0.0       0.0      0.08       0.0   
2      Kosice  Optimal        0.00       0.0       0.0      1.00       0.0   
3      Presov  Optimal        0.48       0.0       0.0      0.00       0.0   
4      Poprad  Optimal        0.00       0.0       0.0      0.00       0.0   
5         ala  Optimal        0.56       0.0       0.0      0.00       0.0   
6          ba  Optimal        0.43       0.0       0.0      0.00       0.0   
7          ca  Optimal        0.00       0.0      -0.0      0.00       0.0   

   Weight_4  Weight_5  Weight_6  Weight_7  scalingFactor_b_0  \
0      0.05       0.0       0.0      1.19                1.0   
1      0.00       0.0       0.0      0.83                1.0   
2     -0.00       0.0       0.0      0.00                0.0   
3      0.07       0.0       0.0      0.74

In [4]:
X = pd.DataFrame(
    np.array(
        [
            [20, 300],
            [30, 200],
            [40, 100],
            [20, 200],
            [10, 400],
            [11, 222],
            [12, 321],
            [14, 231],
        ]
    )
)
y = pd.DataFrame(np.array([[20], [30], [40], [30], [50], [21], [32], [42]]))
b = pd.DataFrame(np.array([[10], [20], [10], [10], [10], [12], [-2], [-1]]))

X,y,b
weight = [1 / 6, 1 / 6, 1 / 3, 1 / 3]
names = pd.DataFrame(
    ["Bratislava", "Zilina", "Kosice", "Presov", "Poprad", "ala", "ba", "ca"],
    columns=["DMU"],
)

solve = DEAProblem(X, y, b, weight, disp="strong disposability").solve()

status = pd.DataFrame.from_dict(solve[0], orient="index", columns=["status"])
efficiency = pd.DataFrame.from_dict(solve[1], orient="index", columns=["efficiency"])
weights = pd.DataFrame.from_dict(solve[2], orient="index")
results = pd.concat([names, status, efficiency, weights], axis=1)
print(results)

          DMU   status    efficiency  Weight_0      Weight_1      Weight_2  \
0  Bratislava  Optimal  9.181842e-01       0.0  0.000000e+00  7.284244e-02   
1      Zilina  Optimal  5.317460e-01       0.0  0.000000e+00  5.267857e-01   
2      Kosice  Optimal  3.333333e-01       0.0  0.000000e+00  1.000000e+00   
3      Presov  Optimal  4.663439e-01       0.0  0.000000e+00  8.298755e-02   
4      Poprad  Optimal  3.333333e-01       0.0  0.000000e+00  0.000000e+00   
5         ala  Optimal  5.785208e-01       0.0  0.000000e+00  0.000000e+00   
6          ba  Optimal  4.840426e-01       0.0  0.000000e+00  0.000000e+00   
7          ca  Optimal  6.223753e-12       0.0 -3.980545e-14  4.257388e-13   

   Weight_3      Weight_4  Weight_5  Weight_6  Weight_7  scalingFactor_b_0  \
0       0.0  4.592241e-02       0.0       0.0  1.187649       1.000000e+00   
1       0.0  0.000000e+00       0.0       0.0  0.637755       1.000000e+00   
2       0.0  0.000000e+00       0.0       0.0  0.000000       1

In [5]:
X,y,b

(    0    1
 0  20  300
 1  30  200
 2  40  100
 3  20  200
 4  10  400
 5  11  222
 6  12  321
 7  14  231,
     0
 0  20
 1  30
 2  40
 3  30
 4  50
 5  21
 6  32
 7  42,
     0
 0  10
 1  20
 2  10
 3  10
 4  10
 5  12
 6  -2
 7  -1)

In [6]:
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno
# 颜色
color = sns.color_palette()
# 数据print精度 ‪C:\Users\52811\Desktop\示例数据.xlsx
pd.set_option('precision',2)
data = pd.read_excel('C:/Users/52811/Desktop/示例数据.xlsx')

In [7]:
#pip install missingno

In [8]:
data

,id,year,labor,cap,energy,gdp,co2
0,1,2010,4912.3,11545.80,46.48,50.07,88.08
1,2,2011,5028.3,12900.80,51.40,57.13,93.94
2,3,2012,5056.0,14125.90,55.22,64.06,98.31
3,4,2013,5137.7,15227.30,59.04,72.38,102.72
4,5,2014,5443.8,16282.10,62.85,82.87,108.56
...,...,...,...,...,...,...,...
131,132,2013,767.9,152.84,11.35,12.63,23.88
132,133,2014,793.2,160.27,12.33,14.11,25.80
133,134,2015,813.0,167.40,13.59,16.36,26.12
134,135,2016,851.3,175.20,16.01,18.33,32.45


In [9]:
data.iloc[:,2:5] 

,labor,cap,energy
0,4912.3,11545.80,46.48
1,5028.3,12900.80,51.40
2,5056.0,14125.90,55.22
3,5137.7,15227.30,59.04
4,5443.8,16282.10,62.85
...,...,...,...
131,767.9,152.84,11.35
132,793.2,160.27,12.33
133,813.0,167.40,13.59
134,851.3,175.20,16.01


In [10]:
col_1=data.iloc[:,2:5] 
col_1
arrs1=col_1.values
arrs1
# DataFrame转化为array
# df=df.values
# array 转化为DataFrame
# df=pd.DataFrame(df)

array([[4.91230e+03, 1.15458e+04, 4.64800e+01],
       [5.02830e+03, 1.29008e+04, 5.14000e+01],
       [5.05600e+03, 1.41259e+04, 5.52200e+01],
       [5.13770e+03, 1.52273e+04, 5.90400e+01],
       [5.44380e+03, 1.62821e+04, 6.28500e+01],
       [5.70260e+03, 1.74333e+04, 6.32700e+01],
       [6.19350e+03, 1.83731e+04, 6.57000e+01],
       [6.46600e+03, 1.92812e+04, 6.95400e+01],
       [6.85900e+03, 2.02552e+04, 6.99500e+01],
       [7.17370e+03, 2.11625e+04, 7.17800e+01],
       [7.42260e+03, 2.19670e+04, 6.72400e+01],
       [7.55860e+03, 2.26826e+04, 6.83100e+01],
       [7.77340e+03, 2.32443e+04, 6.85300e+01],
       [7.91520e+03, 2.37340e+04, 6.96200e+01],
       [1.91020e+03, 1.17001e+03, 3.21500e+01],
       [1.93910e+03, 1.31934e+03, 3.69700e+01],
       [1.94120e+03, 1.45553e+03, 4.08500e+01],
       [1.95000e+03, 1.57875e+03, 4.50000e+01],
       [2.00220e+03, 1.69582e+03, 4.94300e+01],
       [2.00610e+03, 1.82674e+03, 5.36400e+01],
       [2.01650e+03, 1.93572e+03, 5.8740

In [11]:
data.iloc[:,5] 

0      50.07
1      57.13
2      64.06
3      72.38
4      82.87
       ...  
131    12.63
132    14.11
133    16.36
134    18.33
135    20.00
Name: gdp, Length: 136, dtype: float64

In [12]:
col_2=data.iloc[:,5] 
col_2
arrs2=col_2.values
arrs2

array([ 50.072,  57.132,  64.06 ,  72.384,  82.873,  90.422,  99.638,
       109.937, 118.841, 128.03 , 137.889, 147.954, 158.163, 168.909,
        25.78 ,  29.854,  34.302,  39.344,  45.443,  52.941,  61.676,
        72.407,  84.282,  95.955, 107.949, 118.783, 129.835, 141.65 ,
        66.942,  76.448,  85.163,  95.979, 110.568, 121.293, 131.239,
       144.756, 156.626, 168.303, 181.263, 193.951, 207.411, 221.723,
       158.446, 181.896, 207.598, 238.323, 273.833, 302.312, 331.636,
       373.09 , 410.399, 443.87 , 481.598, 518.988, 560.484, 602.54 ,
        69.213,  78.141,  88.612, 100.486, 113.349, 124.797, 137.276,
       154.024, 171.429, 187.933, 203.344, 216.561, 231.287, 247.015,
        60.025,  67.709,  76.308,  87.143, 100.215, 113.644, 128.531,
       146.782, 164.69 , 180.411, 196.107, 207.481, 213.706, 208.363,
       124.429, 142.844, 163.557, 187.926, 215.928, 243.35 , 273.526,
       308.264, 342.173, 376.837, 413.013, 448.945, 487.254, 525.26 ,
        97.05 , 111.

In [13]:
data.iloc[:,6]

0       88.08
1       93.94
2       98.31
3      102.72
4      108.56
        ...  
131     23.88
132     25.80
133     26.12
134     32.45
135     33.66
Name: co2, Length: 136, dtype: float64

In [14]:
col_3=data.iloc[:,6]
col_3
arrs3=col_3.values
arrs3

array([ 88.0812,  93.9385,  98.3134, 102.716 , 108.562 , 103.419 ,
       104.133 , 110.152 ,  98.3035, 100.58  ,  96.3244,  95.4993,
        92.9418,  90.2646,  73.1305,  84.9624,  97.9025, 104.295 ,
       112.293 , 120.35  , 133.236 , 145.343 , 164.538 , 169.659 ,
       175.599 , 173.896 , 167.418 , 161.143 , 142.213 , 156.027 ,
       166.258 , 171.445 , 181.954 , 185.483 , 185.822 , 207.726 ,
       207.85  , 201.321 , 223.338 , 208.673 , 209.356 , 207.8   ,
       256.051 , 298.489 , 345.854 , 377.904 , 413.124 , 424.481 ,
       447.286 , 487.657 , 546.704 , 535.067 , 518.538 , 520.698 ,
       521.522 , 543.584 , 358.886 , 404.734 , 499.78  , 520.477 ,
       564.059 , 587.284 , 607.929 , 749.118 , 810.234 , 826.612 ,
       945.83  , 918.783 , 921.017 , 948.972 , 263.16  , 279.503 ,
       309.135 , 352.127 , 398.09  , 404.198 , 441.663 , 508.956 ,
       498.373 , 505.861 , 556.455 , 563.141 , 546.943 , 539.204 ,
       256.953 , 324.863 , 414.753 , 461.929 , 492.082 , 520.8

In [15]:
X = pd.DataFrame(arrs1)
y = pd.DataFrame(arrs2)
b = pd.DataFrame(arrs3)
X,y,b

(          0         1      2
 0    4912.3  11545.80  46.48
 1    5028.3  12900.80  51.40
 2    5056.0  14125.90  55.22
 3    5137.7  15227.30  59.04
 4    5443.8  16282.10  62.85
 ..      ...       ...    ...
 131   767.9    152.84  11.35
 132   793.2    160.27  12.33
 133   813.0    167.40  13.59
 134   851.3    175.20  16.01
 135   900.8    182.61  16.88
 
 [136 rows x 3 columns],
          0
 0    50.07
 1    57.13
 2    64.06
 3    72.38
 4    82.87
 ..     ...
 131  12.63
 132  14.11
 133  16.36
 134  18.33
 135  20.00
 
 [136 rows x 1 columns],
           0
 0     88.08
 1     93.94
 2     98.31
 3    102.72
 4    108.56
 ..      ...
 131   23.88
 132   25.80
 133   26.12
 134   32.45
 135   33.66
 
 [136 rows x 1 columns])

In [16]:
weight = [1/9,1/9,1/9,1/3,1/3]
solve = DEAProblem(X, y, b, weight, disp="weak disposability").solve()
status = pd.DataFrame.from_dict(solve[0], orient="index", columns=["status"])
efficiency = pd.DataFrame.from_dict(solve[1], orient="index", columns=["efficiency"])
weights = pd.DataFrame.from_dict(solve[2], orient="index")
results = pd.concat([names, status, efficiency, weights], axis=1)

In [18]:
weight = [1/9,1/9,1/9,1/3,1/3]
names = pd.DataFrame(
    ["Bratislava", "Zilina", "Kosice", "Presov", "Poprad", "ala", "ba", "ca"],#####对象名称
    columns=["DMU"],
)
solve = DEAProblem(X, y, b, weight, disp="weak disposability").solve()
status = pd.DataFrame.from_dict(solve[0], orient="index", columns=["status"])
efficiency = pd.DataFrame.from_dict(solve[1], orient="index", columns=["efficiency"])
weights = pd.DataFrame.from_dict(solve[2], orient="index")
results = pd.concat([names, status, efficiency, weights], axis=1)


In [19]:
results.head(50)

,DMU,status,efficiency,Weight_0,Weight_1,Weight_10,Weight_100,Weight_101,Weight_102,Weight_103,...,Weight_95,Weight_96,Weight_97,Weight_98,Weight_99,scalingFactor_b_0,scalingFactor_x_0,scalingFactor_x_1,scalingFactor_x_2,scalingFactor_y_0
0,Bratislava,Optimal,4.60e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,2.45e-01,7.11e-02,0.00e+00,0.00e+00,1.11e+00
1,Zilina,Optimal,4.22e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,2.15e-01,0.00e+00,1.86e-02,0.00e+00,1.04e+00
2,Kosice,Optimal,3.79e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,3.42e-01,0.00e+00,0.00e+00,1.39e-01,7.47e-01
3,Presov,Optimal,3.39e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,4.36e-01,1.16e-02,0.00e+00,2.43e-01,4.97e-01
4,Poprad,Optimal,3.12e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,5.92e-01,2.87e-01,2.85e-01,4.57e-01,0.00e+00
5,ala,Optimal,2.82e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,5.33e-01,2.57e-01,2.71e-01,4.11e-01,0.00e+00
6,ba,Optimal,2.58e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,4.89e-01,2.46e-01,2.38e-01,3.75e-01,0.00e+00
7,ca,Optimal,2.39e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,4.67e-01,2.03e-01,1.99e-01,3.48e-01,0.00e+00
8,NaN,Optimal,1.92e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,3.54e-01,1.88e-01,1.76e-01,3.00e-01,0.00e+00
9,NaN,Optimal,1.71e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00e+00,0.0,0.0,3.20e-01,1.64e-01,1.50e-01,2.65e-01,0.00e+00


In [20]:
df_out_NDDF=results

In [21]:
df_out_NDDF.to_csv('df_out_NDDF2_ln.csv',header=True,index=False)